# Messaging


### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
from libs.messaging_service import MessagingService
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://hlgf-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Instatiate Messaging Service

In [4]:
messaging_service = MessagingService(agent_controller)

## Optional - Configure Mediator

In [5]:
mediator_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'bb47c3ea-ab9a-4ef4-984d-409cb4909956', 'recipientKeys': ['HkwRkbCGEXn4LviXNen5f8kqQ1WxVRpQQGxaQG7PuSZD'], 'serviceEndpoint': 'http://139.162.224.50:6666', 'label': 'HGF_ACA_Playground_Mediator'}

In [6]:
invitation_response = await agent_controller.connections.receive_invitation(mediator_invite)

Connection with ID: 44df4e92-a75a-4f06-a480-3ce287af7131 is now active.
No contact for active connection with ID 44df4e92-a75a-4f06-a480-3ce287af7131


In [7]:
connection_id = invitation_response["connection_id"]
### check state of connection
connection = await agent_controller.connections.get_connection(connection_id)

# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection")    
else:
    ## request mediation
    mediation_req = await agent_controller.mediation.request_mediation(connection_id)
    

In [9]:
### To have a look at the mediation records and we should see our mediation in there
records_response = await agent_controller.mediation.get_mediation_records()
print(f"State : {records_response[0]['state']}")

State : granted


In [10]:
default_mediation_res = await agent_controller.mediation.set_default_mediator(records_response[0]['mediation_id'])
print(default_mediation_res)

{'state': 'granted', 'connection_id': '44df4e92-a75a-4f06-a480-3ce287af7131', 'routing_keys': ['BQKTbHwJap7Su9ZtkBJgqnHEAQt4ysjygBkAEjEngMfp'], 'role': 'client', 'created_at': '2021-06-04 16:22:20.902074Z', 'recipient_terms': [], 'mediation_id': 'e8cabfa4-298d-489c-8707-ad96161c23c4', 'endpoint': 'http://139.162.224.50:6666', 'mediator_terms': [], 'updated_at': '2021-06-04 16:22:21.341659Z'}


## Add New Contacts

### Either Create an Invitation and Share

In [ ]:
alias = "Demo Participant"
response = messaging_service.new_contact_invite(alias)

# If you want to display as Q.R for example
invitation = response["invitation"]

new_contact_id = response["connection_id"]

### Or Add Contact using Invitation You have been sent

In [ ]:
invitation = {"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "b960ab69-f1e2-4c51-ba54-ac2c14ec818b", "recipientKeys": ["3vkzw5qfyzvDgt5bVE6Keeiimz6VuktrXYhGgKeKdKGA"], "label": "Wip", "serviceEndpoint": "https://3a4b8a786a65.ngrok.io"}

In [ ]:
new_contact_id = messaging_service.accept_contact_invitation(invitation, "Demo Participant")

## Send Message

In [ ]:
some_arbitrary_sized_message = "Hello"
messaging_service.send_message(new_contact_id, some_arbitrary_sized_message)

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()